In [17]:
import numpy as np
import pandas as pd
import re
import yfinance as yf
import pandas_market_calendars as mcal

Loading data

In [2]:
convictions = pd.read_csv('Assets\CONVICTIONSUMMARY_270_opt3_best2(1).csv', sep='|') 

In [3]:
convictions = convictions.loc[convictions['TYPE'] == 'MERGED']

Extracting Time and Data

In [ ]:
convictions["Time"] = convictions.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)

convictions["DATE"] = convictions.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)

Cleaning and organizing

In [6]:
## formating data
convictions['DATE'] = pd.to_datetime(
                          convictions['DATE'],
                          format='%Y-%m-%d')

In [7]:
convictions = convictions.sort_values(by="DATE")

In [8]:
# renaming columns
convictions.columns = ['DATE', 'ID', 'STOCK',"SECTOR","TYPE","SCORE","TIME"]

In [9]:
# replace 0 with na in 'SCORE' column
convictions['SCORE']=convictions['SCORE'].replace(0, np.nan)

Adding Returns

In [10]:
list_of_companies = convictions["STOCK"].unique()
list_of_dates = convictions["DATE"].unique()

In [11]:
start_date = convictions.DATE.min()
end_date = convictions.DATE.max()

In [12]:
final_date = end_date + pd.DateOffset(months=12)
print(final_date)

2005-07-14 00:00:00


In [19]:
Columns_names = ['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn']

In [13]:
# extract information about historical prices of stocks for given time period and stock
def get_history_data(ticker_symbol, start_date, end_date):
    tickerData = yf.Ticker(ticker_symbol)
    tickerDf = tickerData.history(period='1d', start=start_date, end=end_date)
    tickerDf["STOCK"] = ticker_symbol

    return tickerDf

In [14]:
# find the trading day which is the same as current date or the closest future day
def get_closest_trading_day(day,trading_days):
    trading_days = trading_days[trading_days>=pd.to_datetime(day, utc=True)]
    return trading_days.sort_values()[0]

In [18]:
# get list of all trading dates between and start and end date on particular market
def get_traning_days(start_date, end_date,stock_market_name):
    nyse = mcal.get_calendar(stock_market_name)
    valid_days = nyse.valid_days(start_date=start_date, end_date=end_date)
    return valid_days

In [73]:
#extracting close price on given data and formating it in case of non such data

def get_price_on_date(date,df):
  try:
    x = df.loc[date]
    x = float(x['Close'])
  except:
    x = 0.0
  return x

In [70]:
# calculate returns for given company for given time aka one row in main dataframe
def calculate_returns(start_date, df, trading_days):
    returns = pd.DataFrame(columns= Columns_names)
    
    # get closes trading dates for specify periods
    date1m = get_closest_trading_day(start_date + pd.DateOffset(months=1),trading_days)
    date3m = get_closest_trading_day(start_date + pd.DateOffset(months=3),trading_days)
    date6m = get_closest_trading_day(start_date + pd.DateOffset(months=6),trading_days)
    date12m = get_closest_trading_day(start_date + pd.DateOffset(months=12),trading_days)

    # if there is no data on the start date return empty data frame
    name = df['STOCK']
    try:
        current = df.loc[start_date]
    except:
        return returns
    
    # get prices
    data1m = get_price_on_date(date1m,df)
    data3m = get_price_on_date(date3m,df)
    data6m = get_price_on_date(date6m,df)
    data12m = get_price_on_date(date12m,df)

    # calculate arithmetic returns
    current = float(current['Close'])

    return1m = (data1m - current)/current
    return3m = (data3m - current)/current
    return6m = (data6m - current)/current
    return12m = (data12m - current)/current

    # to dataframe
    returns = pd.DataFrame([[start_date,name[0],return1m, return3m, return6m,return12m]], columns= Columns_names)

 

    return returns

In [25]:
def calculate_returns_for_date(list_of_dates,df,trading_days):
    returns = pd.DataFrame(columns=Columns_names)
    for d in list_of_dates:
        temp = calculate_returns(pd.to_datetime(d),df,trading_days)
        
        returns = returns.append(temp, ignore_index=True)
    
    return returns

In [68]:
def calculate_returns_for_companies(list_of_dates,list_of_companies,start_date, end_date):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn'])
    trading_days = get_traning_days(start_date,end_date, "NYSE")
    for c in list_of_companies:
        c = c.replace(".XX1","")
        c = c.replace(".XX2","")
        data = get_history_data(c,start_date, end_date)
        len_of_data = data.shape[0]
        if len_of_data !=0:
            returns_for_company = calculate_returns_for_date(list_of_dates,data,trading_days)
            returns = returns.append(returns_for_company, ignore_index=True)
        
    return returns

Downloading Data

In [35]:
trading_days = get_traning_days(start_date, final_date,"NYSE")

In [36]:
trading_days

DatetimeIndex(['2004-02-11 00:00:00+00:00', '2004-02-12 00:00:00+00:00',
               '2004-02-13 00:00:00+00:00', '2004-02-17 00:00:00+00:00',
               '2004-02-18 00:00:00+00:00', '2004-02-19 00:00:00+00:00',
               '2004-02-20 00:00:00+00:00', '2004-02-23 00:00:00+00:00',
               '2004-02-24 00:00:00+00:00', '2004-02-25 00:00:00+00:00',
               ...
               '2005-06-30 00:00:00+00:00', '2005-07-01 00:00:00+00:00',
               '2005-07-05 00:00:00+00:00', '2005-07-06 00:00:00+00:00',
               '2005-07-07 00:00:00+00:00', '2005-07-08 00:00:00+00:00',
               '2005-07-11 00:00:00+00:00', '2005-07-12 00:00:00+00:00',
               '2005-07-13 00:00:00+00:00', '2005-07-14 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=359, freq=None)

In [ ]:
companies_returns = calculate_returns_for_companies(list_of_dates, list_of_companies,start_date, final_date)

In [80]:
companies_returns.head(100)

,DATE,STOCK,1MReturn,3MReturn,6MReturn,12MReturn
0,2004-02-11 00:00:00,SU,0.017659,-0.083391,0.052675,0.416997
1,2004-02-18 00:00:00,SU,0.071475,-0.078276,0.077998,0.479119
2,2004-02-25 00:00:00,SU,0.03862,-0.04447,0.089185,0.572897
3,2004-03-03 00:00:00,SU,-0.01419,-0.107854,0.085261,0.47845
4,2004-03-10 00:00:00,SU,0.002599,-0.093852,0.062814,0.433562
...,...,...,...,...,...,...
95,2004-03-03 00:00:00,APA,-0.002836,-0.028006,0.095016,0.509062
96,2004-03-10 00:00:00,APA,0.052862,0.016214,0.141106,0.488864
97,2004-03-17 00:00:00,APA,0.05063,0.019324,0.122391,0.468461
98,2004-03-24 00:00:00,APA,0.061662,0.08787,0.226167,0.48152


saving dowloaded data

In [82]:
companies_returns.to_csv(r'C:\Users\Krzysztof\Desktop\Universe2\Studia\Przedmioty\ML w finansach\projekt\Finance_ML\Assets\returns.csv')

merging and saving

In [86]:
## formating data
companies_returns['DATE'] = pd.to_datetime(
                          companies_returns['DATE'],
                          format='%Y-%m-%d')

In [87]:
convictions_and_returns = pd.merge(convictions, companies_returns,  how='right', left_on=['DATE','STOCK'], right_on = ['DATE','STOCK'])

In [88]:
convictions_and_returns.head()

,DATE,ID,STOCK,SECTOR,TYPE,SCORE,TIME,1MReturn,3MReturn,6MReturn,12MReturn
0,2004-02-11,GN63J3-R,SU,Energy Minerals,MERGED,0.953727,10:01:54.391,0.017659,-0.083391,0.052675,0.416997
1,2004-02-18,GN63J3-R,SU,Energy Minerals,MERGED,0.953686,10:01:55.714,0.071475,-0.078276,0.077998,0.479119
2,2004-02-25,GN63J3-R,SU,Energy Minerals,MERGED,0.940502,10:01:56.304,0.03862,-0.04447,0.089185,0.572897
3,2004-03-03,GN63J3-R,SU,Energy Minerals,MERGED,0.943390,10:01:57.217,-0.01419,-0.107854,0.085261,0.47845
4,2004-03-10,GN63J3-R,SU,Energy Minerals,MERGED,0.942921,10:01:58.759,0.002599,-0.093852,0.062814,0.433562


In [89]:
convictions_and_returns.to_csv(r'C:\Users\Krzysztof\Desktop\Universe2\Studia\Przedmioty\ML w finansach\projekt\Finance_ML\Assets\convictions_and_returns.csv')